In [207]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

### Load Data

In [182]:
# Load Data
df = pd.read_csv("data/Euro_Salary.csv")

# Create target variable of total compensation
df["Bonus"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Bonus"].fillna(0, inplace=True)
df["target"] = df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Bonus"]
df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR", "Bonus"}, inplace=True)

# Remove rows with more than 7 NaN values
NaN_threshold = 7
df = df[df.isnull().sum(axis=1) <= NaN_threshold]

### Data Cleaning/Prep

In [183]:
# Finding the mode for "Gender" and "Company size" and using that to fill NaN values in those columns
gender_mode = df['Gender'].mode()[0] 
df['Gender'].fillna(gender_mode, inplace=True)

company_mode = df['Company size'].mode()[0] 
df['Company size'].fillna(company_mode, inplace=True)

# Replacing low frequency values with "Other"
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

# Using low frequency function on each categorical column, then filling NaN values with "Other"
df["Seniority level"] = replace_low_freq(df, "Seniority level", 5, "Other")
df["Seniority level"].fillna("Other", inplace=True)

df["City"] = replace_low_freq(df, "City", 10, "Other")
df["City"].fillna("Other", inplace=True)

df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df["Your main technology / programming language"].fillna("Other", inplace=True)

df["Other technologies/programming languages you use often"] = replace_low_freq(df, "Other technologies/programming languages you use often", 10, "Other")
df["Other technologies/programming languages you use often"].fillna("Other", inplace=True)

df["Position "] = replace_low_freq(df, "Position ", 10, "Other")
df["Position "].fillna("Other", inplace=True)

df["Company type"] = replace_low_freq(df, "Company type", 10, "Other")
df["Company type"].fillna("Other", inplace=True)

df["Main language at work"] = replace_low_freq(df, "Main language at work", 10, "Other")
df["Main language at work"].fillna("Other", inplace=True)

df["Employment status"] = replace_low_freq(df, "Employment status", 3, "Other")
df["Employment status"].fillna("Other", inplace=True)

df["Contract duration"] = replace_low_freq(df, "Contract duration", 3, "Other")
df["Contract duration"].fillna("Other", inplace=True)

In [184]:
# Using an imputer to take the mean of the numerical columns for NaN values
mean_columns = ["Age", "Total years of experience", "Years of experience in Germany", "Number of vacation days"]
imputer = SimpleImputer(strategy='mean')

# Replacing non-numerical symbols in data
replacement = {",":".", "<":" "}
df['Total years of experience'] = df['Total years of experience'].replace(replacement, regex=True)
df['Years of experience in Germany'] = df['Years of experience in Germany'].replace(replacement, regex=True)

# Converting column values to numerical to remove words
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], errors='coerce')
df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], errors='coerce')
df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], errors='coerce')

# Calculate and apply mean of column
df[mean_columns] = imputer.fit_transform(df[mean_columns])

In [185]:
# Ordinal encoding for "Seniority level"
seniority_order = {'Other': 0, 'Junior': 1, 'Middle': 2, 'Senior': 3, 'Lead': 4, 'Head': 5}
df['Seniority level'] = df['Seniority level'].map(seniority_order)

### Feature Selection

In [152]:
# Feature selection using Variance Threshold
df_vf = pd.get_dummies(df, drop_first=True)
y = df_vf["target"]
X = df_vf.drop(columns={"target"})

from sklearn.feature_selection import VarianceThreshold
var_th = VarianceThreshold(.2)
print(X.shape)
post_vt = var_th.fit_transform(X)
print(post_vt.shape)

mask = var_th.get_support()
new_features = X.columns[mask]
print(new_features)

(1247, 67)
(1247, 9)
Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'Seniority level', 'Number of vacation days',
       'Position _Software Engineer',
       'Your main technology / programming language_Other',
       'Company size_101-1000', 'Company type_Product'],
      dtype='object')


In [199]:
# New data frame after results of variance threshold feature selection
df_fs1 = df.copy()
df_fs1.drop(columns={"Gender", "Other technologies/programming languages you use often", "Employment status", "Main language at work", "Contract duration"}, inplace=True)

### Model Testing

In [208]:
# Preparing data with one hot encoding
#df1 = pd.get_dummies(df, drop_first=True)

#y = df1["target"]
#X = df1.drop(columns={"target"})

y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))

xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size=.3)

numeric_features = ["Age", "Total years of experience", "Years of experience in Germany", "Number of vacation days"]  # List of column names with continuous numerical features
categorical_features = ["Gender", "City", "Position ", "Seniority level", "Your main technology / programming language", "Other technologies/programming languages you use often",
                        "Employment status", "Contract duration", "Main language at work", "Company size", "Company type"]  # List of column names with categorical features

# Create a transformer that applies MinMaxScaler to numerical features
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Create a transformer that applies OneHotEncoder to categorical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [196]:
#Train model with Linear Regression
model = LinearRegression().fit(xTrain,yTrain)
print("Training R2:", model.score(xTrain,yTrain))

#RMSE with test data
model_preds = model.predict(xTest)
print("Test RMSE:", mean_squared_error(model_preds,yTest,squared=False))
print("Test R2:", model.score(xTest,yTest))

Training R2: 0.17213609824355047
Test RMSE: 79681473.0977757
Test R2: -1747145.1946782991


In [138]:
# One hot encoding for all other categorical columns
df_vf1 = pd.get_dummies(df_fs1, drop_first=True)
y_vf = np.array(df_vf1["target"]).reshape(-1,1)
X_vf = np.array(df_vf1.drop(columns={"target"}))

x_vfTrain,x_vfTest,y_vfTrain,y_vfTest = train_test_split(X_vf,y_vf,test_size=.3)

#Train model with Linear Regression
model_vf = LinearRegression().fit(x_vfTrain,y_vfTrain)
print("Training R2:", model_vf.score(x_vfTrain,y_vfTrain))

#RMSE with test data
model_vf_preds = model_vf.predict(x_vfTest)
print("RMSE test:", mean_squared_error(model_vf_preds,y_vfTest,squared=False))
print("R2 test:", model_vf.score(x_vfTest,y_vfTest))

Training R2: 0.39573745030280283
RMSE test: 422729069.66154927
R2 test: -81547709.90535146


In [210]:
#Regression Tree Model

scalar = MinMaxScaler()
regression_tree = DecisionTreeRegressor()
pipe = Pipeline(steps=[(preprocessor), ("regression_tree", regression_tree)])

tree_para = {'regression_tree__min_samples_split':[2,3,4,5,6,7],
            'regression_tree__max_depth':[7,8,9,10,11,12]}

model = GridSearchCV(pipe, param_grid=tree_para, cv=5, n_jobs=-1)
model.fit(xTrain, yTrain)
model_tr = model.best_estimator_


print(model_tr)

print("Train Score:", model.score(xTrain, yTrain))
model_preds = model.predict(xTest)
print("RMSE:", mean_squared_error(model_preds,yTest,squared=False))
print("R2:", np.mean(cross_val_score(model_tr, xTrain, yTrain.ravel(), cv=5)))
print("Test score:", model.score(xTest, yTest))



ValueError: Invalid parameter 'regression_tree' for estimator Pipeline(steps=[ColumnTransformer(transformers=[('num',
                                                 Pipeline(steps=[('scaler',
                                                                  MinMaxScaler())]),
                                                 ['Age',
                                                  'Total years of experience',
                                                  'Years of experience in '
                                                  'Germany',
                                                  'Number of vacation days']),
                                                ('cat',
                                                 Pipeline(steps=[('onehot',
                                                                  OneHotEncoder())]),
                                                 ['Gender', 'City', 'Position ',
                                                  'Seniority level',
 

In [206]:
#Standard Gradient Descent Regression Model

scalar = MinMaxScaler()
regression_tree = SGDRegressor(max_iter=2000)
pipe = Pipeline(steps=[("scalar", scalar), ("regression_tree", regression_tree)])

pipe.fit(xTrain, yTrain.ravel())



#print(model_sgd)

print("Train Score:", pipe.score(xTrain, yTrain))
model_preds = pipe.predict(xTest)
print("RMSE:", mean_squared_error(model_preds,yTest,squared=False))
print("R2:", np.mean(cross_val_score(pipe, xTrain, yTrain.ravel(), cv=5)))
print("Test score:", pipe.score(xTest, yTest))

Train Score: 0.14830265526050623
RMSE: 71393328.110718
R2: -2225523.622377915
Test score: -890125.310682618


In [200]:
df2 = pd.get_dummies(df_fs1, drop_first=True)

#y = df1["target"]
#X = df1.drop(columns={"target"})

y = np.array(df2["target"]).reshape(-1,1)
X = np.array(df2.drop(columns={"target"}))

xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size=.3)

# Answers and Explanations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes